In [ ]:
# Cell 1: Imports and Setup
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from scipy import fftpack
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

In [ ]:
# Cell 2: Define paths - Update these based on your dataset structure
base_path = Path('/home/tigerit/Documents/face-liveliness/casia-fasd/test')

# Example: 5 real + 5 spoof from 2 subjects (s1 and s2)
real_images = [
    base_path / 'live/s1v1f0.png',
    base_path / 'live/s1v1f1.png',
    base_path / 'live/s1v1f2.png',
    base_path / 'live/s1v2f0.png',
    base_path / 'live/s1v2f1.png',
    base_path / 'live/s2v1f0.png',
    base_path / 'live/s2v1f1.png',
    base_path / 'live/s2v1f2.png',
    base_path / 'live/s2v2f0.png',
    base_path / 'live/s2v2f1.png',
]

spoof_images = [
    base_path / 'spoof/s1v3f0.png',
    base_path / 'spoof/s1v3f1.png',
    base_path / 'spoof/s1v3f2.png',
    base_path / 'spoof/s1v4f0.png',
    base_path / 'spoof/s1v4f1.png',
    base_path / 'spoof/s2v3f0.png',
    base_path / 'spoof/s2v3f1.png',
    base_path / 'spoof/s2v3f2.png',
    base_path / 'spoof/s2v4f0.png',
    base_path / 'spoof/s2v4f1.png',
]

# Filter only existing files
real_images = [p for p in real_images if p.exists()][:5]
spoof_images = [p for p in spoof_images if p.exists()][:5]

print(f"Found {len(real_images)} real images")
print(f"Found {len(spoof_images)} spoof images")

In [ ]:
# Cell 3: Function to compute 2D FFT and extract features
def compute_fft_features(image_path, visualize=False):
    """
    Compute 2D FFT of an image and extract frequency-domain features
    """
    # Read image in grayscale
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        print(f"Could not read: {image_path}")
        return None
    
    # Resize for consistency
    img = cv2.resize(img, (256, 256))
    
    # Apply 2D FFT
    f_transform = np.fft.fft2(img)
    f_shift = np.fft.fftshift(f_transform)
    
    # Magnitude spectrum
    magnitude_spectrum = np.abs(f_shift)
    
    # Log scale for better visualization
    magnitude_spectrum_log = np.log1p(magnitude_spectrum)
    
    # Phase spectrum
    phase_spectrum = np.angle(f_shift)
    
    if visualize:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(img, cmap='gray')
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        axes[1].imshow(magnitude_spectrum_log, cmap='jet')
        axes[1].set_title('Magnitude Spectrum (Log Scale)')
        axes[1].axis('off')
        
        axes[2].imshow(phase_spectrum, cmap='hsv')
        axes[2].set_title('Phase Spectrum')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return {
        'original': img,
        'fft': f_shift,
        'magnitude': magnitude_spectrum,
        'magnitude_log': magnitude_spectrum_log,
        'phase': phase_spectrum
    }

In [ ]:
# Cell 4: Visualize examples from both classes
print("=" * 50)
print("REAL FACE EXAMPLE")
print("=" * 50)
real_example = compute_fft_features(real_images[0], visualize=True)

print("\n" + "=" * 50)
print("SPOOF FACE EXAMPLE")
print("=" * 50)
spoof_example = compute_fft_features(spoof_images[0], visualize=True)

In [ ]:
# Cell 6: Average magnitude spectrum comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Average real
avg_real = np.mean([r['magnitude_log'] for r in real_ffts], axis=0)
im1 = axes[0].imshow(avg_real, cmap='jet')
axes[0].set_title('Average Magnitude Spectrum - REAL', fontsize=14, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], fraction=0.046)

# Average spoof
avg_spoof = np.mean([s['magnitude_log'] for s in spoof_ffts], axis=0)
im2 = axes[1].imshow(avg_spoof, cmap='jet')
axes[1].set_title('Average Magnitude Spectrum - SPOOF', fontsize=14, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], fraction=0.046)

# Difference
diff = avg_real - avg_spoof
im3 = axes[2].imshow(diff, cmap='RdBu_r', vmin=-diff.std()*2, vmax=diff.std()*2)
axes[2].set_title('Difference (Real - Spoof)', fontsize=14, fontweight='bold')
axes[2].axis('off')
plt.colorbar(im3, ax=axes[2], fraction=0.046)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 7: Radial frequency analysis (key discriminator!)
def compute_radial_profile(magnitude_spectrum):
    """
    Compute radial average of frequency spectrum
    This shows energy distribution across frequencies
    """
    h, w = magnitude_spectrum.shape
    center = (h // 2, w // 2)
    
    # Create distance map from center
    y, x = np.ogrid[:h, :w]
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2).astype(int)
    
    # Compute radial average
    max_r = min(center)
    radial_prof = np.zeros(max_r)
    
    for radius in range(max_r):
        mask = (r == radius)
        radial_prof[radius] = magnitude_spectrum[mask].mean() if mask.any() else 0
    
    return radial_prof

# Compute radial profiles
real_profiles = [compute_radial_profile(r['magnitude']) for r in real_ffts]
spoof_profiles = [compute_radial_profile(s['magnitude']) for s in spoof_ffts]

# Plot
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
for i, prof in enumerate(real_profiles):
    plt.plot(prof, alpha=0.6, label=f'Real {i+1}', color='green')
plt.plot(np.mean(real_profiles, axis=0), 'g-', linewidth=3, label='Real Average')
plt.xlabel('Frequency (radius from center)', fontsize=12)
plt.ylabel('Magnitude', fontsize=12)
plt.title('Radial Frequency Profile - REAL', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
for i, prof in enumerate(spoof_profiles):
    plt.plot(prof, alpha=0.6, label=f'Spoof {i+1}', color='red')
plt.plot(np.mean(spoof_profiles, axis=0), 'r-', linewidth=3, label='Spoof Average')
plt.xlabel('Frequency (radius from center)', fontsize=12)
plt.ylabel('Magnitude', fontsize=12)
plt.title('Radial Frequency Profile - SPOOF', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 8: Direct comparison of real vs spoof profiles
plt.figure(figsize=(15, 5))

# Log scale comparison
plt.subplot(1, 2, 1)
real_avg = np.mean(real_profiles, axis=0)
spoof_avg = np.mean(spoof_profiles, axis=0)

plt.plot(real_avg, 'g-', linewidth=2.5, label='Real (Average)', alpha=0.8)
plt.plot(spoof_avg, 'r-', linewidth=2.5, label='Spoof (Average)', alpha=0.8)
plt.xlabel('Frequency (radius from center)', fontsize=12)
plt.ylabel('Magnitude', fontsize=12)
plt.title('Real vs Spoof - Radial Frequency Profile', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

# Log scale
plt.subplot(1, 2, 2)
plt.semilogy(real_avg, 'g-', linewidth=2.5, label='Real (Average)', alpha=0.8)
plt.semilogy(spoof_avg, 'r-', linewidth=2.5, label='Spoof (Average)', alpha=0.8)
plt.xlabel('Frequency (radius from center)', fontsize=12)
plt.ylabel('Magnitude (log scale)', fontsize=12)
plt.title('Real vs Spoof - Log Scale', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 9: Extract frequency-based features for classification
def extract_frequency_features(fft_data):
    """
    Extract discriminative features from FFT
    """
    magnitude = fft_data['magnitude']
    h, w = magnitude.shape
    center = (h // 2, w // 2)
    
    # Radial profile
    radial_prof = compute_radial_profile(magnitude)
    
    # High frequency energy (outer ring)
    outer_radius = int(min(center) * 0.8)
    inner_radius = int(min(center) * 0.5)
    
    y, x = np.ogrid[:h, :w]
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    
    high_freq_mask = (r >= inner_radius) & (r <= outer_radius)
    low_freq_mask = r < inner_radius * 0.3
    
    high_freq_energy = magnitude[high_freq_mask].sum()
    low_freq_energy = magnitude[low_freq_mask].sum()
    total_energy = magnitude.sum()
    
    features = {
        'high_freq_energy': high_freq_energy,
        'low_freq_energy': low_freq_energy,
        'high_low_ratio': high_freq_energy / (low_freq_energy + 1e-10),
        'high_freq_percentage': (high_freq_energy / total_energy) * 100,
        'radial_profile_mean': radial_prof.mean(),
        'radial_profile_std': radial_prof.std(),
        'radial_profile_slope': np.polyfit(range(len(radial_prof)), radial_prof, 1)[0],
    }
    
    return features

# Extract features
real_features = [extract_frequency_features(r) for r in real_ffts]
spoof_features = [extract_frequency_features(s) for s in spoof_ffts]

# Create comparison dataframe
import pandas as pd

real_df = pd.DataFrame(real_features)
real_df['class'] = 'Real'

spoof_df = pd.DataFrame(spoof_features)
spoof_df['class'] = 'Spoof'

combined_df = pd.concat([real_df, spoof_df], ignore_index=True)

print("\n" + "=" * 60)
print("FEATURE STATISTICS")
print("=" * 60)
print("\nREAL FACES:")
print(real_df.describe())
print("\nSPOOF FACES:")
print(spoof_df.describe())